In [ ]:
%load_ext autoreload
%autoreload 2

import json
from pathlib import Path
from nerfiller.experiments.data import occluder_dataset_names

In [ ]:
def get_metrics(datasets, method_name, dataset_inpaint_method, metrics_list):
    """Given a list of datasets, compute the average metrics for each dataset."""
    metrics = [[] for _ in metrics_list]
    for dataset in datasets:
        # print(dataset, method_name)
        filename = str(sorted(list(Path("../outputs/" + dataset + "-" + dataset_inpaint_method + "/" + method_name).iterdir()))[-1]) + "/final-renders/test-metrics/test.json"
        # print(filename)
        with open(filename, "r") as f:
            metrics_dict = json.load(f)
            for idx, metric_name in enumerate(metrics_list):
                metrics[idx].append(metrics_dict[metric_name])
    # get the mean
    metrics = [sum(metric) / len(metric) for metric in metrics]
    return metrics

In [ ]:
TEMPLATE = \
"""
\\begin{table}[]
    \\captionsetup{font=scriptsize}
    \\scriptsize
    \\small
    \\begin{tabular}{l|lll} % the number of columns needed
        \\toprule
        COLUMNS \\\\
        \\midrule
        EXPERIMENT_LINES
        \\bottomrule
    \\end{tabular}
    \\caption{\\textbf{CAPTION_TITLE.} CAPTION_TEXT}
    \\label{tab:TABLE_LABEL}
\\end{table}
"""

# TEMPLATE = \
# """
# \\begin{table}[]
#     \\captionsetup{font=scriptsize}
#     \\scriptsize
#     \\footnotesize
#     \\begin{tabular}{l|lll|ll} % the number of columns needed
#         \\toprule
#         COLUMNS \\\\
#         \\midrule
#         EXPERIMENT_LINES
#         \\bottomrule
#     \\end{tabular}
#     \\caption{\\textbf{CAPTION_TITLE.} CAPTION_TEXT}
#     \\label{tab:TABLE_LABEL}
# \\end{table}
# """

In [ ]:
# # SYNTHETIC EXPERIMENTS
# methods are the columns
# here we map the title in the column to the the method name (which we can grab from folder names)
# COLUMNS = "\n & PSNR $\\uparrow$ & SSIM $\\uparrow$ & LPIPS $\\downarrow$"
# METRICS_LIST = ["psnr", "ssim", "lpips"]
# SIGFIGS = [2, 2, 2]
# assert len(METRICS_LIST) == len(SIGFIGS)
# METHODS = [
#     ("nerfacto-nerfiller", "none", "Masked NeRF"),
#     ("nerfacto-nerfiller", "individual-lama", "LaMask"),
#     ("nerfacto-nerfiller", "individual-sd-text", "SD Text Cond"),
#     ("nerfacto-nerfiller", "individual-sd-image", "SD Image Cond"),
#     ("nerfacto-nerfiller", "expanded-attention", "Extended Attention []"),
#     ("nerfacto-nerfiller", "grid-prior-no-joint", "Grid Prior"),
#     ("nerfacto-nerfiller", "grid-prior", "Multi-Grid Prior"),
# ]
# DATASETS = ["chair", "drums", "ficus", "hotdog", "lego", "materials", "mic", "ship"]
# TABLE_LABEL = "2d_inpainting_consistency"
# caption_title = "2D inpainting consistency"
# caption_text = """
# We use various inpainting methods to inpaint images once and then train a NeRF.
# These are the average metrics reported over all 8 scenes of the NeRF synthetic dataset.
# Better metrics indicate more consistency of the 3D reconstruction with the 2D inpaints.
# The Grid Prior method achieves the most consistent results. Please see supplementary material for video rendering results.
# """

# 3D OCCLUDER EXPERIMENTS
# methods are the columns
# here we map the title in the column to the the method name (which we can grab from folder names)
COLUMNS = "\n & PSNR $\\uparrow$ & SSIM $\\uparrow$ & LPIPS $\\downarrow$ & MUSIQ $\\uparrow$ & Corrs $\\uparrow$"
METRICS_LIST = ["psnr", "ssim", "lpips", "musiq", "corrs"]
SIGFIGS = [2, 2, 2, 2, 0]
assert len(METRICS_LIST) == len(SIGFIGS)
METHODS = [
    ("nerfacto-nerfiller", "none", "Masked NeRF"),
    ("individual-inpaint-once", "individual-lama", "LaMask"),
    ("individual-inpaint-once", "individual-sd-image", "SD Image Cond"),
    ("individual-inpaint-du", "none", "Inpaint + DU"),
    ("grid-prior-du-no-depth", "none", "Ours w/o depth"),
    ("grid-prior-du", "none", "Ours"),
]
DATASETS = list(occluder_dataset_names)
TABLE_LABEL = "quantitative_nerf_baselines"
caption_title = "Quantitative NeRF baselines"
caption_text = """
Here we compare with various methods implemented within the Nerfstudio framework.
We report metrics computed in the unknown mask region to quantify consistency of the 3D inpainted region.
"""

def print_datasets(datasets, caption_title="caption title", TABLE_LABEL="label", caption_text="caption text"):
    # metrics = get_metrics(datasets, "nerfacto-visibility-sparsity", METRICS_LIST)
    # metrics = [str(f"{metric:.{sigfigs}f}") for metric, sigfigs in zip(metrics, SIGFIGS)]

    # create the experiment lines
    EXPERIMENT_LINES = ["\n"]
    for idx, (method_name, dataset_inpaint_method, table_name) in enumerate(METHODS):
        # if idx == 1:
        #     line_string = f"\\midrule \n"
        #     EXPERIMENT_LINES.append(line_string)
        metrics = get_metrics(datasets, method_name, dataset_inpaint_method, METRICS_LIST)
        metrics = [str(f"{metric:.{sigfigs}f}") for metric, sigfigs in zip(metrics, SIGFIGS)]
        line_string = f"{table_name} & {' & '.join(metrics)} \\\\ \n"
        EXPERIMENT_LINES.append(line_string)

    s = TEMPLATE.replace("COLUMNS", COLUMNS)
    s = s.replace("EXPERIMENT_LINES", " ".join(EXPERIMENT_LINES))
    s = s.replace("CAPTION_TITLE", caption_title)
    s = s.replace("CAPTION_TEXT", caption_text)
    s = s.replace("TABLE_LABEL", TABLE_LABEL)
    print(s)


# for the average of all the datasets
print_datasets(DATASETS, caption_title=caption_title, TABLE_LABEL=TABLE_LABEL, caption_text=caption_text)

# print("---" * 10)

In [ ]:
# for all the datasets individually
for i in range(len(DATASETS)):
    caption_title_indiv = f"{caption_title} for data ``{DATASETS[i]}\""
    print_datasets(DATASETS[i : i + 1], caption_title=caption_title_indiv, TABLE_LABEL=DATASETS[i], caption_text="")